Install & Import Libraries

In [1]:
#installs
!pip install --upgrade pip
!pip install --upgrade transformers datasets scikit-learn pandas numpy

#imports
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import accuracy_score, f1_score, classification_report
import pandas as pd
import numpy as np
import json
from sklearn.feature_extraction.text import TfidfVectorizer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

Load Custom Dataset

In [2]:
#custom train and test data
train_data = [
    ("Terrible quality, broke in a week.", 1),
    ("It is okay, does the job.", 3),
    ("Fantastic service, very happy with purchase!", 5),
    ("Average product, nothing special.", 3),
    ("Excellent quality and works perfectly.", 4),
    ("Horrible experience, will never buy again.", 1),
    ("Satisfactory product, nothing to complain.", 3),
    ("Superb item, highly recommend it!", 5),
    ("Completely useless, total waste of money.", 1),
    ("Product works fine, but not outstanding.", 3),
    ("Really good performance, totally worth it.", 4),
    ("Very poor build, extremely fragile.", 1),
    ("Decent for the price, acceptable quality.", 3),
    ("Very satisfied, exceeded expectations.", 5),
    ("Bad product, nothing like advertised.", 1),
    ("Neutral experience, exactly what I expected.", 3),
    ("Great product and fast delivery!", 4),
    ("Worst experience, terrible quality overall.", 1),
    ("Top-notch quality, impressed with the build.", 5),
    ("Cheap materials, feels very low quality.", 2),
    ("Not bad, but could be better.", 3),
    ("Amazing purchase, would buy again.", 4),
    ("I regret buying this, awful performance.", 1),
    ("Quality is fine, nothing extraordinary.", 3),
    ("Perfectly meets all my needs, excellent!", 5),
    ("Extremely disappointing, do not recommend.", 1),
    ("Reasonable item for the cost.", 3),
    ("Usable product, neither good nor bad.", 3),
    ("Great product and fast delivery!", 5),
    ("Really good performance, totally worth it.", 5)
]

test_data = [
    ("Loved this item, great purchase", 5),
    ("Worst product ever", 1),
    ("It is fine, meets expectations", 3),
    ("Very happy with this purchase", 5),
    ("Disappointing quality", 2),
    ("Average item, nothing extraordinary", 3),
    ("Excellent product, exceeded my expectations", 4),
    ("Very poor build, broke immediately", 1),
    ("Product is okay, neither good nor bad", 3),
    ("The quality is fantastic, worth the price", 4),
    ("Not good, completely useless", 1),
    ("Works decently, acceptable for the price", 3),
    ("Super useful and works perfectly", 5),
    ("Terrible experience, do not buy", 1),
    ("Satisfactory overall, but could be better", 3)
]

df_train = pd.DataFrame(train_data, columns=["review", "stars"])
df_test  = pd.DataFrame(test_data, columns=["review", "stars"])

def map_star_to_label(star):
    if star <= 2:
        return 0   # Negative
    elif star == 3:
        return 1   # Neutral
    else:
        return 2   # Positive

df_train["label"] = df_train["stars"].apply(map_star_to_label)
df_test["label"] = df_test["stars"].apply(map_star_to_label)


Tokenizer & Dataset Class

In [3]:

model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

#Dataset class
class ReviewDataset(Dataset):
    def __init__(self, df):
        self.texts = df["review"].tolist()
        self.labels = df["label"].tolist()

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        encoding = tokenizer(
            self.texts[idx],
            truncation=True,
            padding="max_length",
            max_length=64,
            return_tensors="pt"
        )
        item = {key: val.squeeze(0) for key, val in encoding.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item
train_ds = ReviewDataset(df_train)
test_ds = ReviewDataset(df_test)

train_loader = DataLoader(train_ds, batch_size=8, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=8, shuffle=False)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Load Model & Optimizer

In [4]:
#load model
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=3
)
model.to(device)

optimizer = AdamW(model.parameters(), lr=5e-5)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Training Loop

In [5]:
#training
model.train()

for epoch in range(3):
    total_loss = 0

    for batch in train_loader:
        for k in batch:
            batch[k] = batch[k].to(device)

        outputs = model(**batch)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1} | Loss = {total_loss:.4f}")


Epoch 1 | Loss = 4.3506
Epoch 2 | Loss = 4.0882
Epoch 3 | Loss = 3.2709


Evaluation (Accuracy + F1)

In [6]:
model.eval()

all_preds = []
all_labels = []

with torch.no_grad():
    for batch in test_loader:
        for k in batch:
            batch[k] = batch[k].to(device)

        outputs = model(**batch)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(batch["labels"].cpu().numpy())


accuracy = accuracy_score(all_labels, all_preds)
f1 = f1_score(all_labels, all_preds, average="macro")

print("===== MODEL PERFORMANCE =====")
print("Accuracy:", accuracy)
print("F1 Score:", f1)


===== MODEL PERFORMANCE =====
Accuracy: 0.8666666666666667
F1 Score: 0.8611111111111112


Error Analysis

In [7]:
print("\n===== ERROR ANALYSIS =====")

errors = []

for i, (pred, actual) in enumerate(zip(all_preds, all_labels)):
    if pred != actual:
        errors.append({
            "review": df_test.iloc[i]["review"],
            "actual_label": actual,
            "predicted_label": pred
        })

if len(errors) == 0:
    print("No misclassifications found in test set.")
else:
    for err in errors:
        print("\nMisclassified Review:")
        print("Review Text:", err["review"])
        print("Actual Label:", err["actual_label"])
        print("Predicted Label:", err["predicted_label"])
        print("---------------------------")


===== ERROR ANALYSIS =====

Misclassified Review:
Review Text: It is fine, meets expectations
Actual Label: 1
Predicted Label: 2
---------------------------

Misclassified Review:
Review Text: Satisfactory overall, but could be better
Actual Label: 1
Predicted Label: 2
---------------------------


Sentiment + Summary Generator

In [9]:
vectorizer = TfidfVectorizer(max_features=3000, stop_words="english")
vectorizer.fit(df_test["review"])


def extract_top_phrases_tfidf(review, vectorizer, top_k=3):
    vec = vectorizer.transform([review])
    indices = vec.toarray()[0].argsort()[::-1]
    feature_names = vectorizer.get_feature_names_out()

    top_phrases = []
    for idx in indices:
        word = feature_names[idx]
        if word.isalpha():
            top_phrases.append(word)
        if len(top_phrases) == top_k:
            break

    return top_phrases

summary = []

label_map = {0: "Negative", 1: "Neutral", 2: "Positive"}

for review in df_test["review"]:

    encoding = tokenizer(
        review,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=64
    ).to(device)

    with torch.no_grad():
        logits = model(**encoding).logits
        pred = torch.argmax(logits, dim=1).item()
        confidence = torch.softmax(logits, dim=1)[0][pred].item()

    supporting_phrases = extract_top_phrases_tfidf(review, vectorizer)

    follow_up = (
        "Consider improving product quality or addressing customer complaints."
        if pred == 0 else None
    )

    summary.append({
        "review_text": review,
        "sentiment": {
            "predicted": label_map[pred],
            "confidence_score": round(confidence, 4),
            "supporting_phrases": supporting_phrases,
            "follow_up_suggestion": follow_up
        }
    })


with open("structured_summary.json", "w") as f:
    json.dump(summary, f, indent=2)

print("\nStructured Summary:")
print(json.dumps(summary[:5], indent=2))



Structured Summary:
[
  {
    "review_text": "Loved this item, great purchase",
    "sentiment": {
      "predicted": "Positive",
      "confidence_score": 0.4934,
      "supporting_phrases": [
        "loved",
        "great",
        "item"
      ],
      "follow_up_suggestion": null
    }
  },
  {
    "review_text": "Worst product ever",
    "sentiment": {
      "predicted": "Negative",
      "confidence_score": 0.5134,
      "supporting_phrases": [
        "worst",
        "product",
        "works"
      ],
      "follow_up_suggestion": "Consider improving product quality or addressing customer complaints."
    }
  },
  {
    "review_text": "It is fine, meets expectations",
    "sentiment": {
      "predicted": "Positive",
      "confidence_score": 0.4413,
      "supporting_phrases": [
        "meets",
        "fine",
        "expectations"
      ],
      "follow_up_suggestion": null
    }
  },
  {
    "review_text": "Very happy with this purchase",
    "sentiment": {
      "pred